# Credit Risk Pipeline Quickstart

This notebook runs the **Unified Risk Pipeline** end-to-end on the bundled synthetic dataset.
The sample includes stratified monthly observations, calibration hold-outs, stage-2 data, and a future scoring batch
so each major step can be validated quickly.


## 1. Environment & Data Preparation

In [ ]:
import importlib
import importlib.metadata as metadata
import importlib.util
import subprocess
import sys
from pathlib import Path


def _locate_project_root() -> Path:
    cwd = Path.cwd().resolve()
    if (cwd / 'src' / 'risk_pipeline').exists():
        return cwd
    candidate = cwd / 'risk-model-pipeline-dev'
    if (candidate / 'src' / 'risk_pipeline').exists():
        return candidate
    for parent in cwd.parents:
        maybe = parent / 'risk-model-pipeline-dev'
        if (maybe / 'src' / 'risk_pipeline').exists():
            return maybe
    return cwd


PROJECT_ROOT = _locate_project_root()
SRC_PATH = PROJECT_ROOT / 'src'
PACKAGE_PATH = SRC_PATH / 'risk_pipeline'
MODULE_INIT = PACKAGE_PATH / '__init__.py'
if SRC_PATH.exists() and str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

TARGET_VERSION = '0.4.1'
GIT_SPEC = 'risk-pipeline[ml,notebook] @ git+https://github.com/selimoksuz/risk-model-pipeline.git@development'
PREREQ_PACKAGES = [
    'numba==0.59.1',
    'llvmlite==0.42.0',
    'scipy==1.11.4',
    'pandas==2.3.2',
    'tsfresh==0.20.1',
    'matrixprofile==1.1.10',
    'shap==0.48.0',
    'stumpy==1.13.0',
]


def _parse_version(value: str):
    parts = []
    for part in value.split('.'):
        if not part.isdigit():
            break
        parts.append(int(part))
    return tuple(parts)


def _run_pip(args):
    subprocess.check_call([
        sys.executable,
        '-m',
        'pip',
        'install',
        '--no-cache-dir',
        '--upgrade',
        '--force-reinstall',
        *args,
    ])


def _install_prerequisites():
    print(f"Installing prerequisite stack: {', '.join(PREREQ_PACKAGES)}")
    _run_pip(PREREQ_PACKAGES)


def _sanity_check():
    import shap  # noqa: F401
    from llvmlite import binding as _ll_binding
    _ = _ll_binding.ffi.lib
    from numba import njit

    @njit
    def _probe(x):
        return x + 1

    assert _probe(1) == 2


def _tsfresh_smoke_test():
    import pandas as pd
    from tsfresh import extract_features
    from tsfresh.feature_extraction import EfficientFCParameters

    data = pd.DataFrame(
        {
            'id': ['a', 'a', 'a', 'b', 'b', 'b'],
            'time': [0, 1, 2, 0, 1, 2],
            'value': [1.0, 2.0, 3.0, 4.0, 9.0, 16.0],
        }
    )
    features = extract_features(
        data,
        column_id='id',
        column_sort='time',
        column_value='value',
        default_fc_parameters=EfficientFCParameters(),
        disable_progressbar=True,
        n_jobs=0,
    )
    if not any('entropy' in col for col in features.columns):
        raise RuntimeError('tsfresh smoke test did not produce entropy features')


def _resolve_installed_version(module):
    module_path = Path(getattr(module, '__file__', '')).resolve()
    if SRC_PATH in module_path.parents:
        return TARGET_VERSION
    try:
        return metadata.version('risk-pipeline')
    except metadata.PackageNotFoundError:
        return '0.0.0'


def _load_local_package():
    if not MODULE_INIT.exists():
        return None
    spec = importlib.util.spec_from_file_location('risk_pipeline', MODULE_INIT)
    if spec and spec.loader:
        module = importlib.util.module_from_spec(spec)
        sys.modules['risk_pipeline'] = module
        spec.loader.exec_module(module)
        return module
    return None


def ensure_risk_pipeline():
    print(f"Resolved project root: {PROJECT_ROOT}")
    try:
        module = _load_local_package()
        if module is None:
            module = importlib.import_module('risk_pipeline')
        installed = _resolve_installed_version(module)
        if _parse_version(installed) < _parse_version(TARGET_VERSION):
            raise ModuleNotFoundError(f'risk-pipeline {installed} < {TARGET_VERSION}')
        print(f'risk-pipeline {installed} available (path: {module.__file__}).')
        _sanity_check()
        _tsfresh_smoke_test()
    except Exception as exc:
        print(f'risk-pipeline import failed: {exc}')
        try:
            _install_prerequisites()
            print(f'Attempting GitHub install: {GIT_SPEC}')
            _run_pip([GIT_SPEC])
            print('GitHub install succeeded.')
            raise SystemExit('Installation complete. Restart the kernel and rerun this cell.')
        except subprocess.CalledProcessError as err:
            print(f'GitHub install failed: {err}')
            raise SystemExit('Installation failed. Review the errors above.')
    else:
        print('Numba/llvmlite sanity check passed.')
        print('tsfresh smoke test passed (entropy features available).')


ensure_risk_pipeline()


In [ ]:

from pathlib import Path
import pandas as pd

from IPython.display import display

from risk_pipeline.data.sample import load_credit_risk_sample

sample = load_credit_risk_sample()
OUTPUT_DIR = Path('output/credit_risk_sample_notebook')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dev_df = sample.development.copy()
cal_long_df = sample.calibration_longrun.copy()
cal_recent_df = sample.calibration_recent.copy()
score_df = sample.scoring_future.copy()
data_dictionary = sample.data_dictionary.copy()

print(f"Development dataset: {dev_df.shape[0]:,} rows, {dev_df.shape[1]} columns")
print(f"Stage 1 calibration dataset: {cal_long_df.shape[0]:,} rows")
print(f"Stage 2 calibration dataset: {cal_recent_df.shape[0]:,} rows")
print(f"Scoring dataset: {score_df.shape[0]:,} rows")

display(dev_df.head())


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

from risk_pipeline.core.config import Config
from risk_pipeline.unified_pipeline import UnifiedRiskPipeline
from risk_pipeline.data.sample import load_credit_risk_sample

sample = load_credit_risk_sample()
OUTPUT_DIR = Path('output/credit_risk_sample_notebook')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dev_df = sample.development
cal_long_df = sample.calibration_longrun
cal_recent_df = sample.calibration_recent
score_df = sample.scoring_future
data_dictionary = sample.data_dictionary

dev_df.head()


In [ ]:
import importlib

import risk_pipeline.core.config as config_module
import risk_pipeline.unified_pipeline as pipeline_module

Config = importlib.reload(config_module).Config
UnifiedRiskPipeline = importlib.reload(pipeline_module).UnifiedRiskPipeline

cfg_params = {
    # Core identifiers
    'target_column': 'target',
    'id_column': 'customer_id',
    'time_column': 'app_dt',

    # Split configuration
    'create_test_split': True,
    'stratify_test': True,
    'train_ratio': 0.8,
    'test_ratio': 0.2,
    'oot_ratio': 0.0,
    'oot_size': 0.0,
    'oot_months': 3,

    # TSFresh controls
    'enable_tsfresh_features': True,
    'tsfresh_feature_set': 'efficient',
    'tsfresh_n_jobs': 4,

    # Feature selection strategy
    'selection_steps': [
        'univariate',
        'psi',
        'vif',
        'correlation',
        'iv',
        'boruta',
        'stepwise',
    ],
    'min_univariate_gini': 0.05,
    'psi_threshold': 0.25,
    'monthly_psi_threshold': 0.15,
    'oot_psi_threshold': 0.25,
    'max_vif': 5.0,
    'correlation_threshold': 0.9,
    'iv_threshold': 0.02,
    'stepwise_method': 'forward',
    'stepwise_max_features': 25,

    # Model training preferences
    'algorithms': [
        'logistic',
        'lightgbm',
        'xgboost',
        'catboost',
        'randomforest',
        'extratrees',
        'woe_boost',
        'woe_li',
        'shao',
        'xbooster',
    ],
    'model_selection_method': 'gini_oot',
    'model_stability_weight': 0.2,
    'min_gini_threshold': 0.5,
    'max_train_oot_gap': 0.03,
    'use_optuna': True,
    'hpo_trials': 75,
    'hpo_timeout_sec': 1800,

    # Diagnostics & toggles
    'use_noise_sentinel': True,
    'enable_dual': True,
    'enable_woe_boost_scorecard': True,
    'calculate_shap': True,
    'enable_scoring': True,
    'enable_stage2_calibration': True,

    # Risk band settings
    'n_risk_bands': 10,
    'risk_band_method': 'pd_constraints',
    'risk_band_min_bins': 7,
    'risk_band_max_bins': 10,
    'risk_band_hhi_threshold': 0.15,
    'risk_band_binomial_pass_weight': 0.85,

    # Runtime controls
    'random_state': 42,
    'n_jobs': -1,
}

cfg_field_names = set(Config.__dataclass_fields__.keys())
supported_params = {k: v for k, v in cfg_params.items() if k in cfg_field_names}
unsupported = sorted(set(cfg_params.keys()) - set(supported_params.keys()))
if unsupported:
    print(f"[WARN] Config ignores unsupported parameters: {unsupported}")

cfg = Config(**supported_params)

pipe = UnifiedRiskPipeline(cfg)


## 3. TSFresh Feature Extraction

In [ ]:

processed = pipe.run_process(dev_df, create_map=True, force=True)
print(f"Processed feature space: {processed.shape[1]} columns")

if pipe.data_.get('tsfresh_metadata') is not None and not pipe.data_['tsfresh_metadata'].empty:
    display(pipe.data_['tsfresh_metadata'].head())
else:
    print('No TSFresh features were generated (configuration disabled).')


## 4. Raw Numeric Processing

In [ ]:

splits = pipe.run_split(processed, force=True)
raw_layers = pipe.results_.get('raw_numeric_layers', {})
print(f"Identified numeric features: {len(pipe.data_.get('numeric_features', []))}")
if raw_layers:
    train_raw = raw_layers.get('train_raw_prepped')
    if train_raw is not None:
        display(train_raw[pipe.data_.get('numeric_features', [])].head())
else:
    print('No numeric preprocessing layer was created.')

impute_stats = getattr(pipe.data_processor, 'imputation_stats_', {})
if impute_stats:
    display(pd.DataFrame(impute_stats).T.head())

# Summarise configuration choices for quick inspection
config_summary = pd.DataFrame([
    ("Target column", cfg.target_column),
    ("ID column", cfg.id_column),
    ("Time column", cfg.time_column),
    ("Train/Test/OOT split", f"{cfg.train_ratio:.0%}/{cfg.test_ratio:.0%}/{cfg.oot_ratio:.0%}"),
    ("OOT holdout months", cfg.oot_months),
    ("Risk bands", f"{cfg.n_risk_bands} (method={cfg.risk_band_method})"),
    ("Calibration chain", f"{cfg.calibration_stage1_method} -> {cfg.calibration_stage2_method}"),
], columns=["Parameter", "Configured value"])
display(config_summary)

flag_toggles = pd.DataFrame({
    "Feature": [
        "Dual RAW+WOE flow",
        "TSFresh feature mining",
        "Scoring on hold-out data",
        "Stage 2 calibration",
        "Optuna HPO",
        "Noise sentinel",
        "SHAP importance",
    ],
    "Enabled": [
        getattr(cfg, 'enable_dual', False),
        getattr(cfg, 'enable_tsfresh_features', False),
        getattr(cfg, 'enable_scoring', False),
        getattr(cfg, 'enable_stage2_calibration', False),
        getattr(cfg, 'use_optuna', False),
        getattr(cfg, 'use_noise_sentinel', False),
        getattr(cfg, 'calculate_shap', False),
    ],
})
flag_toggles['Enabled'] = flag_toggles['Enabled'].map({True: 'Yes', False: 'No'})
display(flag_toggles)

thresholds = pd.DataFrame({
    "Threshold": [
        "PSI",
        "IV",
        "Univariate Gini",
        "Correlation ceiling",
        "VIF ceiling",
        "|Train-OOT| Gini gap",
    ],
    "Value": [
        cfg.psi_threshold,
        cfg.iv_threshold,
        cfg.univariate_gini_threshold,
        cfg.correlation_threshold,
        cfg.vif_threshold,
        cfg.max_train_oot_gap,
    ],
})
display(thresholds)

selection_order = pd.DataFrame({"Selection step": cfg.selection_steps})
selection_order.index = selection_order.index + 1
display(selection_order)

algorithms_df = pd.DataFrame({"Algorithm": cfg.algorithms})
algorithms_df.index = algorithms_df.index + 1
display(algorithms_df)


## 5. WOE Transformation

In [ ]:

woe_results = pipe.run_woe(splits, force=True)
woe_values = woe_results.get('woe_values', {})
print(f"WOE maps generated for {len(woe_values)} variables")
if woe_values:
    preview = pd.DataFrame([
        {
            'variable': name,
            'type': info.get('type'),
            'iv': info.get('iv'),
        }
        for name, info in list(woe_values.items())[:5]
    ])
    display(preview)


## 6. Feature Selection

### 5.1 Raw vs prepped numeric diagnostics


In [ ]:
processed_df = pipe.data_.get('processed')
if processed_df is None or processed_df.empty:
    print("Processed dataset snapshot is not available.")
else:
    numeric_cols = (
        dev_df.select_dtypes(include=['number']).columns
        .difference([cfg.target_column])
    )
    diagnostics = []
    for col in numeric_cols:
        raw_series = dev_df[col]
        proc_series = processed_df[col]
        missing_raw = int(raw_series.isna().sum())
        missing_proc = int(proc_series.isna().sum())
        diagnostics.append({
            'feature': col,
            'raw_missing': missing_raw,
            'processed_missing': missing_proc,
            'missing_delta': missing_raw - missing_proc,
            'raw_p01': float(raw_series.quantile(0.01)),
            'proc_p01': float(proc_series.quantile(0.01)),
            'raw_p99': float(raw_series.quantile(0.99)),
            'proc_p99': float(proc_series.quantile(0.99)),
            'raw_mean': float(raw_series.mean()),
            'proc_mean': float(proc_series.mean()),
        })
    diagnostics_df = pd.DataFrame(diagnostics)
    if diagnostics_df.empty:
        print("No numeric columns found for diagnostics.")
    else:
        diagnostics_df['clip_delta_low'] = diagnostics_df['proc_p01'] - diagnostics_df['raw_p01']
        diagnostics_df['clip_delta_high'] = diagnostics_df['proc_p99'] - diagnostics_df['raw_p99']
        diagnostics_df['mean_shift'] = diagnostics_df['proc_mean'] - diagnostics_df['raw_mean']
        display(
            diagnostics_df.sort_values(['missing_delta', 'clip_delta_low'], ascending=[False, False])
            .head(12)
        )
        top_cols = (
            diagnostics_df.sort_values(['missing_delta', 'clip_delta_low'], ascending=[False, False])
            ['feature'].head(4).tolist()
        )
        if top_cols:
            comparison = pd.concat(
                {
                    'raw': dev_df[top_cols],
                    'prepped': processed_df[top_cols]
                }, axis=1
            )
            display(comparison.head(5))
    get_summary = getattr(pipe.splitter, 'get_split_summary', None)
    if callable(get_summary):
        split_summary = get_summary()
        if isinstance(split_summary, pd.DataFrame) and not split_summary.empty:
            display(split_summary)


### 5.2 TSFresh feature contributions


In [ ]:
tsfresh_meta = pipe.reporter.reports_.get('tsfresh_metadata')
if tsfresh_meta is None or tsfresh_meta.empty:
    print("No TSFresh metadata captured (feature generation may be disabled).")
else:
    tsfresh_meta = tsfresh_meta.copy()
    if 'source_variable' in tsfresh_meta.columns:
        tsfresh_meta['source_variable'] = tsfresh_meta['source_variable'].astype(str)
    else:
        tsfresh_meta['source_variable'] = 'unknown'
    if 'statistic' in tsfresh_meta.columns:
        tsfresh_meta['statistic'] = tsfresh_meta['statistic'].fillna('unknown').astype(str)
    else:
        tsfresh_meta['statistic'] = 'unknown'
    total_features = int(tsfresh_meta['feature'].nunique())
    total_bases = int(tsfresh_meta['source_variable'].nunique())
    print(f"Generated {total_features} TSFresh features across {total_bases} base variables.")
    source_summary = (
        tsfresh_meta.groupby('source_variable')
        .size()
        .rename('feature_count')
        .sort_values(ascending=False)
        .reset_index()
    )
    if not source_summary.empty:
        source_summary['share'] = (
            source_summary['feature_count'] / source_summary['feature_count'].sum()
        ).round(4)
        display(source_summary.head(15))
    stat_mix = (
        tsfresh_meta.groupby('statistic')
        .size()
        .rename('feature_count')
        .sort_values(ascending=False)
        .reset_index()
    )
    if not stat_mix.empty:
        display(stat_mix)
    display(tsfresh_meta.head(10))


### 5.3 WOE transformation quality


In [ ]:
woe_results = results.get('woe_results') or {}
if not woe_results:
    print("WOE results dictionary is empty.")
else:
    woe_values = woe_results.get('woe_values', {}) or {}
    gini_map = woe_results.get('univariate_gini', {}) or {}
    summary_rows = []
    for feature, info in woe_values.items():
        stats = info.get('stats') or []
        feature_type = info.get('type', 'unknown')
        if feature_type == 'numeric':
            if info.get('bins') is not None:
                bin_count = max(len(info['bins']) - 1, len(stats))
            else:
                bin_count = len(stats)
        else:
            categories = info.get('categories')
            if categories:
                bin_count = len(categories)
            else:
                bin_count = len(stats)
        gini_info = gini_map.get(feature, {}) or {}
        summary_rows.append({
            'feature': feature,
            'type': feature_type,
            'iv': info.get('iv'),
            'bin_count': bin_count,
            'gini_raw': gini_info.get('gini_raw'),
            'gini_woe': gini_info.get('gini_woe'),
        })
    summary_df = pd.DataFrame(summary_rows)
    if summary_df.empty:
        print("No WOE summary rows to display.")
    else:
        summary_df['gini_uplift'] = summary_df['gini_woe'].fillna(0) - summary_df['gini_raw'].fillna(0)
        display(summary_df.sort_values('iv', ascending=False).head(15))
        display(summary_df.sort_values('gini_uplift').head(10))
        top_feature = summary_df.sort_values('iv', ascending=False)['feature'].iloc[0]
        top_stats = woe_values.get(top_feature, {}).get('stats')
        if top_stats:
            display(pd.DataFrame(top_stats).head(15))


### 5.4 Feature selection progression


In [ ]:
selection_results = results.get('selection_results') or {}
selection_history = pipe.reporter.reports_.get('selection_history')
if selection_history is None or selection_history.empty:
    print("Selection history is empty.")
else:
    selection_history = selection_history.copy()
    if 'details' in selection_history.columns:
        details_series = selection_history['details']
        selection_history = selection_history.drop(columns='details')
    else:
        details_series = pd.Series([{} for _ in range(len(selection_history))])
    expanded = details_series.apply(lambda val: val if isinstance(val, dict) else {}).apply(pd.Series)
    history_df = pd.concat([selection_history, expanded], axis=1)
    display(history_df)
selected_features = selection_results.get('selected_features') or results.get('selected_features') or []
if selected_features:
    feature_list = pd.DataFrame({'rank': np.arange(1, len(selected_features) + 1), 'feature': selected_features})
    display(feature_list.head(40))
    print(f"Total selected features: {len(selected_features)}")
feature_report = pipe.reporter.reports_.get('features')
if isinstance(feature_report, pd.DataFrame) and not feature_report.empty:
    display(feature_report.head(25))
dropped = selection_results.get('dropped_features')
if dropped:
    if isinstance(dropped, dict):
        dropped_df = pd.DataFrame(list(dropped.items()), columns=['feature', 'reason'])
    else:
        dropped_df = pd.DataFrame(dropped)
    if not dropped_df.empty:
        display(dropped_df.head(40))


### 5.5 Model performance comparison


In [ ]:
model_results = results.get('model_results', {})
model_scores = model_results.get('scores', {}) or {}
if not model_scores:
    print("Model scores not available.")
else:
    scores_df = pd.DataFrame(model_scores).T
    metric_priority = [col for col in ['oot_auc', 'test_auc', 'train_auc'] if col in scores_df.columns]
    metric_cols = [col for col in ['train_auc', 'test_auc', 'oot_auc', 'train_gini', 'test_gini', 'oot_gini', 'train_oot_gap'] if col in scores_df.columns]
    display(scores_df[metric_cols].sort_values(by=metric_priority, ascending=False))
    if 'train_auc' in scores_df.columns and any(col in scores_df.columns for col in ['oot_auc', 'test_auc']):
        ref_cols = [col for col in ['oot_auc', 'test_auc'] if col in scores_df.columns]
        scores_df['overfit_gap'] = scores_df['train_auc'] - scores_df[ref_cols].max(axis=1)
        display(scores_df['overfit_gap'].sort_values())
best_model = model_results.get('best_model_name') or results.get('best_model_name')
chosen_flow = results.get('chosen_flow')
if best_model and model_scores:
    print(f"Best model: {best_model} (flow: {chosen_flow})")
    if best_model in model_scores:
        display(pd.Series(model_scores[best_model]).dropna())
feature_importance = model_results.get('feature_importance', {})
if feature_importance and best_model in feature_importance:
    fi_df = pd.DataFrame(feature_importance[best_model])
    if not fi_df.empty:
        display(fi_df.head(25))


### 5.6 Calibration metrics


In [ ]:
stage1 = results.get('calibration_stage1') or {}
stage2 = results.get('calibration_stage2') or {}
stage1_metrics = stage1.get('calibration_metrics')
if stage1_metrics:
    stage1_df = pd.DataFrame(stage1_metrics, index=['Stage 1']).T
    display(stage1_df)
    if stage1.get('stage1_details'):
        display(pd.DataFrame(stage1['stage1_details'], index=['Stage 1']).T)
stage2_metrics = stage2.get('stage2_metrics')
if stage2_metrics:
    stage2_df = pd.DataFrame(stage2_metrics, index=['Stage 2']).T
    display(stage2_df)
stage2_details = stage2.get('stage2_details')
if stage2_details:
    display(pd.DataFrame(stage2_details, index=['Stage 2']).T)
calibration_report = pipe.reporter.reports_.get('calibration')
if calibration_report is not None:
    display(calibration_report)


### 5.7 Risk band optimisation


In [ ]:
risk_band_results = results.get('risk_bands') or {}
if not risk_band_results:
    print("Risk band optimizer did not return results.")
else:
    band_table = risk_band_results.get('bands') or risk_band_results.get('band_table')
    if isinstance(band_table, pd.DataFrame) and not band_table.empty:
        display(band_table)
    metrics = risk_band_results.get('metrics') or {}
    if metrics:
        display(pd.DataFrame(metrics, index=['metrics']).T)
risk_band_summary = pipe.reporter.reports_.get('risk_bands_summary')
if isinstance(risk_band_summary, pd.DataFrame) and not risk_band_summary.empty:
    display(risk_band_summary)


### 5.8 Recent scoring diagnostics


In [ ]:
scoring_output = results.get('scoring_output') or {}
scoring_metrics = scoring_output.get('metrics') or results.get('scoring_metrics') or {}
if scoring_metrics:
    flat_metrics = {
        key: value
        for key, value in scoring_metrics.items()
        if not isinstance(value, (dict, list))
    }
    if flat_metrics:
        summary_df = pd.DataFrame(flat_metrics, index=['value']).T
        display(summary_df)
    if 'with_target' in scoring_metrics and scoring_metrics['with_target']:
        display(pd.DataFrame(scoring_metrics['with_target'], index=['with_target']).T)
    if 'without_target' in scoring_metrics and scoring_metrics['without_target']:
        display(pd.DataFrame(scoring_metrics['without_target'], index=['without_target']).T)
reports = scoring_output.get('reports') or results.get('scoring_reports') or {}
for name, report in reports.items():
    if isinstance(report, pd.DataFrame) and not report.empty:
        print(f"Report: {name}")
        display(report)
scored_df = scoring_output.get('dataframe') or results.get('scoring_results')
if isinstance(scored_df, pd.DataFrame) and not scored_df.empty:
    scored_view = scored_df.copy()
    if 'risk_score' not in scored_view.columns:
        scored_view['risk_score'] = np.nan
    band_summary = scored_view.groupby('risk_band').agg(
        records=('risk_band', 'size'),
        avg_score=('risk_score', 'mean'),
        min_score=('risk_score', 'min'),
        max_score=('risk_score', 'max')
    )
    if 'target' in scored_view.columns:
        band_summary['bads'] = scored_view.groupby('risk_band')['target'].sum()
        band_summary['bad_rate'] = scored_view.groupby('risk_band')['target'].mean()
    display(band_summary)
    display(scored_view.head(10))
else:
    print("Scoring dataframe is not available.")


In [ ]:

stage1 = pipe.run_stage1_calibration(model_results=pipe.results_['model_results'], calibration_df=cal_long_df, force=True)
stage2 = pipe.run_stage2_calibration(stage1_results=stage1, recent_df=cal_recent_df, force=True)

if isinstance(stage1, dict) and stage1.get('calibration_curve') is not None:
    curve = stage1['calibration_curve']
    if hasattr(curve, 'head'):
        display(curve.head())

pipe.results_['calibration_stage1'] = stage1
pipe.results_['calibration_stage2'] = stage2
print('Stage 1 and Stage 2 calibration completed.')


## 9. Risk Band Optimisation

In [ ]:

bands = pipe.run_risk_bands(stage2_results=stage2, splits=pipe.results_['splits'], force=True)
pipe.results_['risk_bands'] = bands

if isinstance(bands, dict):
    metrics = bands.get('metrics')
    if isinstance(metrics, dict):
        display(pd.DataFrame(metrics, index=['value']).T)
    else:
        print('Risk band metrics not available.')


## 10. Consolidated Pipeline Run

In [ ]:

full_pipe = UnifiedRiskPipeline(cfg)
full_results = full_pipe.fit(
    dev_df,
    data_dictionary=data_dictionary,
    calibration_df=cal_long_df,
    stage2_df=cal_recent_df,
    score_df=score_df,
)

print(f"Best mode: {full_results.get('best_model_mode')} | Best model: {full_results.get('best_model_name')}")
print('Model registry (top rows):')
model_registry = pd.DataFrame(full_results.get('model_registry', []))
if not model_registry.empty:
    display(model_registry.sort_values(['mode', 'oot_auc'], ascending=[True, False]).head())
else:
    print('Model registry is empty.')


## 11. Recent Raw Data Scoring

In [ ]:

scoring_output = pipe.run_scoring(score_df, force=True)
scored_df = scoring_output.get('dataframe')
if scored_df is not None:
    display(scored_df.head())
metrics = scoring_output.get('metrics')
if metrics:
    print('Scoring metrics:')
    display(pd.DataFrame(metrics, index=[0]).T)


For automation examples, see examples/quickstart_demo.py.